In [1]:
#!c1.32
import os, time, cv2, sys, math
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as l
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import tqdm
import pandas as pd
import base64
import operator

from Utils.ADE20K_utils import *
from Utils.images_utilities import *


In [2]:
#!c1.32
# from FC_DenseNet_Tiramisu import build_tiramisu
tf.compat.v1.enable_eager_execution()

In [3]:
#!c1.32
# Загрузка индекса с глобальной информацией о датасете
DATASET_PATH = '/home/jupyter/mnt/datasets/ADE20K'
index_file = '/ADE20K_2021_17_01/index_ade20k.pkl'
with open('{}/{}'.format(DATASET_PATH, index_file), 'rb') as f:
    index_ade20k = pkl.load(f)

In [4]:
#!c1.32

# В данном коде идёт загрузка списков путей для дальнейшей обработки. 
# Для оптимизации иполнения был создан отдельный файл, в который закидываются строки.
# Не хотел добавлять новых библиотек, cправился без Pandas, а зря.

train = []
train_seg = []
val = []
val_seg = []
test = []
test_seg = []

file = './Data/file_list.txt'
if os.path.exists(file):
    with open(file, 'r') as f:
        paths_list = f.read()
        paths_list = list(paths_list.replace("'", "").replace('[', '').replace(']', '').split(sep=", "))
        for i in tqdm.tqdm(range(0, 25248), desc='Набор фото тренировки'):
            train.append(paths_list[i])
        for i in tqdm.tqdm(range(25248, 27248), desc='Набор фото валидации'):
            val.append(paths_list[i])
        for i in tqdm.tqdm(range(27248, 27564), desc='Набор фото тестирования'):
            test.append(paths_list[i])
        for i in tqdm.tqdm(range(27564, 52812), desc='Набор сег. фото тренировки'):
            train_seg.append(paths_list[i])
        for i in tqdm.tqdm(range(52812, 54812), desc='Набор сег. фото валидации'):
            val_seg.append(paths_list[i])
        for i in tqdm.tqdm(range(54812, 55128), desc='Набор сег. фото тестирования'):
            test_seg.append(paths_list[i])
        f.close()
else:
    for i in tqdm.tqdm(range(0, 27574), desc="Занесено пар: "):
        full_file_name = '{}/{}'.format(index_ade20k['folder'][i], index_ade20k['filename'][i])
        try:
            info = loadAde20K('{}/{}'.format(DATASET_PATH, full_file_name))
            if 0 <= i <= 25257:
                train.append(info['img_name'])
                train_seg.append(info['segm_name'])
            elif 25258 <= i <= 27257:
                val.append(info['img_name'])
                val_seg.append(info['segm_name'])
            elif 27258 <= i <= 27573:
                test.append(info['img_name'])
                test_seg.append(info['segm_name'])
        except KeyboardInterrupt:
            print('ПРЕРЫВАЮ!')
            break
        except:
            tqdm.tqdm.write('Ошибка на индексе'+str(i))

    with open(file, 'w+') as f:
        f.write(str(train + val + test + train_seg + val_seg + test_seg))
        f.close()

print('Данные были загружены.')

Данные были загружены.


Набор сег. фото тестирования: 100%|██████████| 316/316 [00:00<00:00, 1535805.40it/s]


In [5]:
#!c1.32
def dense_block(input_tensor, block_num, num_layers, growth_rate):
    concat_layers = [input_tensor]
    x = input_tensor
    for i in range(num_layers):
        layer = l.BatchNormalization(name=f'bn_{block_num}_{i}')(x)
        layer = l.ReLU(name=f'relu_{block_num}_{i}')(layer)
        layer = l.Conv2D(growth_rate, (3, 3), padding='same', name=f'conv_{block_num}_{i}')(layer)
        concat_layers.append(layer)
        x = l.concatenate(concat_layers, axis=-1, name=f'concat_{block_num}_{i}')
    return x

def transition_down(input_tensor, block_num):
    x = l.BatchNormalization(name=f'bn_td_{block_num}')(input_tensor)
    x = l.ReLU(name=f'relu_td_{block_num}')(x)
    x = l.Conv2D(int(x.shape[-1]) // 2, (1, 1), name=f'conv_td_{block_num}')(x)
    x = l.MaxPooling2D((2, 2), strides=(2, 2), name=f'maxpool_td_{block_num}')(x)
    return x

def build_tiramisu(input_shape, num_classes, num_blocks=5, num_layers_per_block=4, growth_rate=16):
    input_layer = l.Input(shape=input_shape, name='input')
    x = input_layer
    
    skip_connections = []

    for i in range(num_blocks):
        x = dense_block(x, i+1, num_layers_per_block, growth_rate)
        skip_connections.append(x)
        if i != num_blocks - 1:
            x = transition_down(x, i+1)

    x = dense_block(x, num_blocks+1, num_layers_per_block, growth_rate)

    for i in range(num_blocks - 1, -1, -1):
        x = l.Conv2DTranspose(int(skip_connections[i].shape[-1]) // 2, (3, 3), strides=(2, 2), padding='same')(x)
        x = tf.image.resize(x, skip_connections[i].shape[1:3])
        x = l.concatenate([x, skip_connections[i]], axis=-1)
        x = dense_block(x, num_blocks+2+i, num_layers_per_block, growth_rate)

    output_layer = l.Conv2D(num_classes, (1, 1), activation='softmax', name='output')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    return model

In [6]:
#!c1.32

print('Настройка процедуры обучения...')

network = build_tiramisu((360, 640, 3), 3688)

Настройка процедуры обучения...


2023-08-15 20:42:06.445464: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jupyter/.local/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64
2023-08-15 20:42:06.445519: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-15 20:42:06.445543: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (m2-e58e1947-b749-4503-9869-22f2a47d758b): /proc/driver/nvidia/version does not exist
2023-08-15 20:42:06.445808: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them i

In [7]:
#!c1.32
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=network, labels=output))
# class_names_string = str(index_ade20k['objectnames']).replace('[', '').replace(']', '').replace("'", "")

# Инициализация параметров
epochs = 25
is_training = True
cont_training = False
class_names_list = index_ade20k['objectnames']
loss = tf.keras.losses.CategoricalCrossentropy()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, decay=0.995)

# Создание чекпоинтов модели
checkp_path = './Checkpoints/'
checkpoint = tf.train.Checkpoint(network)
save_path = checkpoint.save(checkp_path)
if cont_training or not is_training:
    print('Загрузка чекпоинта...')
    checkpoint.restore(save_path)



pd_file_path_tr = './Data/images_train.csv'
pd_file_path_v = './Data/images_val.csv'
pd_file_path_t = './Data/images_test.csv'
if not os.path.exists(pd_file_path_tr) and not os.path.exists(pd_file_path_v) and not os.path.exists(pd_file_path_t):
    
    print('Запущен процесс загрузки и кодирования картинок для последующего формирования .csv файлов...')
    
    target_size=(640, 360)
    input_train_images_b64, output_train_images_b64 = [], []
    input_val_images_b64, output_val_images_b64 = [], []
    input_test_images_b64, output_test_images_b64 = [], []
    
    input_train_images_b64, output_train_images_b64 = image_to_base64(load_images(train, train_seg, target_size))
    del train, train_seg
    input_val_images_b64, output_val_images_b64 = image_to_base64(load_images(val, val_seg, target_size))
    del val, val_seg
    input_test_images_b64, output_test_images_b64 = image_to_base64(load_images(test, test_seg, target_size))
    del test, test_seg
    
    data_train = {'inp_tr_img_b64': input_train_images_b64, 'outp_tr_img_b64': output_train_images_b64}
    data_val = {'inp_val_img_b64': input_val_images_b64, 'outp_val_img_b64': output_val_images_b64}
    data_test = {'inp_test_img_b64': input_test_images_b64, 'outp_test_img_b64': output_test_images_b64}
    
    df_train = pd.DataFrame(data_train)
    df_val = pd.DataFrame(data_val)
    df_test = pd.DataFrame(data_test)
    
    df_train.to_csv(pd_file_path_tr, index=False)
    df_val.to_csv(pd_file_path_v, index=False)
    df_test.to_csv(pd_file_path_t, index=False)
    del input_train_images_b64, output_train_images_b64, input_val_images_b64, output_val_images_b64, input_test_images_b64, output_test_images_b64
    del df_train, df_val, df_test
    print('Процес успешно завершён!')
    

print('Запущен процесс загрузки картинок из .csv файла...')
df_train = pd.read_csv(pd_file_path_tr)
df_val = pd.read_csv(pd_file_path_v)
df_test = pd.read_csv(pd_file_path_t)
    
input_train_images, output_train_images = base64_to_img(df_train['inp_tr_img_b64'], df_train['outp_tr_img_b64'])
input_val_images, output_val_images = base64_to_img(df_val['inp_val_img_b64'], df_val['outp_val_img_b64'])
input_test_images, output_test_images = base64_to_img(df_test['inp_test_img_b64'], df_test['outp_test_img_b64'])
print('Успешно завершено!')
del df_train, df_val, df_test

Запущен процесс загрузки картинок из .csv файла...
Успешно завершено!


Декодирую...: 100%|██████████| 316/316 [00:02<00:00, 134.84it/s]


In [8]:
#!c1.32
images_norm = img_normilize(input_train_images[:10])
del input_train_images


Нормализация...: 100%|██████████| 10/10 [00:00<00:00, 179.05it/s]


In [25]:
#!c1.32
mask_path = '/home/jupyter/mnt/datasets/ADE20K/ADE20K_2021_17_01/images/ADE/training/cultural/armory/ADE_train_00001593_seg.png'
mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

# Получение значения класса для конкретного пикселя (x, y)
x = 393  # Координата x пикселя
y = 167  # Координата y пикселя

class_id = mask[y, x]

print(f"Значение класса для пикселя ({x}, {y}): {class_id}")

Значение класса для пикселя (393, 167): [142  68  80]


In [9]:
#!c1.32
log_dir = './Logs/'
callbacks = [
        tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            embeddings_freq=1,)
    ]

images_norm = np.array(images_norm)
masks = np.array(output_train_images[:10])
print(images_norm[1].shape)
print(masks[1].shape)

network.compile(optimizer = opt,
               loss = loss)

# network.summary()
network.fit(images_norm, masks, epochs=epochs, batch_size=64, shuffle=True, callbacks=callbacks)


2023-08-15 20:47:53.054949: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2023-08-15 20:47:53.054997: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2023-08-15 20:47:53.055376: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2023-08-15 20:47:53.400388: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(360, 640, 3)
(360, 640, 3)
Epoch 1/25


ValueError: in user code:

    /usr/local/lib/python3.8/dist-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/dist-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/dist-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /usr/local/lib/python3.8/dist-pack

In [ ]:
#!c1.32
